# Maxwell's Daemon
## Final Project for Computer Based Physical Modelling
### Thomas de Paula Barbosa - 3749592

# About the Daemon
Maxwell's daemon is a thought experiment made to test the validity the Second Law of Thermodynamics. The thought experiment consists in two isolated boxes filled with gas which have a gate in between them. The gate's opening and closing is controlled by a daemon who only allows molecules that have a certain velocity to pass through the gate. After a set amount of time, this means that faster (hotter) molecules will be on one side, while the slower (colder) molecules will be in the other side. Assuming this could actually happen, one of the system's container would decrease in temperature and the other one would increase. Since we would be ordering an unordered system without adding energy, this would actually decrease its entropy, which is a direct violation of the Second Law of Thermodynamics.

## Collisions
In the simulation, all collisions are perfectly elastic and momentum is conserved. Since all the masses are the same, the resulting velocity of each particle is as follows:
$$
P_i = P_f\\
m_1 u_1 + m_2 u_2 = m_1 v_1 + m_2 v_2\\
u_1 + u_2 = v_1 + v_2
$$

As for kinetic energy, we have

$$
K_i = K_f\\
\frac{1}{2}m_1 u_1^2 + \frac{1}{2}m_2 u_2^2 = \frac{1}{2}m_1 v_1^2 + \frac{1}{2}m_2 v_2^2
$$

Solving for $v_1$ and $v_2$ we have

$$
v_1 = \frac{m_1 - m_2}{m_1 + m_2}u_1 + \frac{2m_2}{m_1 + m_2}u_2\\
v_2 = \frac{m_2 - m_1}{m_1 + m_2}u_2 + \frac{2m_1}{m_1 + m_2}u_1
$$

Since both masses are the same, the solution becomes:

$$
v_1 = u_2 \\
v_2 = u_1
$$

In code, this means that we can (and will) simply exchange the velocity of the first particle with the velocity of the second particle.

# Code
The objective of this code is to simulate Maxwell's Daemon and observe what happens to the temperature in each container. Although Python is dynamically typed, I like giving type hints on various bits for keeping track of things more easily. 

## Particle Class

In [1]:
import numpy as np



CONTAINER_COORDINATES = np.array([800, 600])  # x, y
# the daemon will open the gate for a particle when it reaches this speed. This also helps determine whether a particle is hot or cold
SPEED_THRESHHOLD: float = 4.0
GATE_SIZE: int = 100
X, Y = 0, 1  # this is just for ease of reading
PARTICLE_RADIUS: int = 5


class Particle:
    def __init__(self, position: np.ndarray, velocity: np.ndarray, radius: int):
        self.position: np.ndarray = position
        self.velocity: np.ndarray = velocity
        self.radius: int = radius
        self._speed: float = np.linalg.norm(velocity)
        self.isHot: bool = True if abs(self._speed) >= SPEED_THRESHHOLD else False
        self._isStuck: int = 0

    def __repr__(self):
        return f"Particle({self.position}, {self.velocity}, {self.radius}, {self._speed}, {self.isHot})"

    def _calculateIsHot(self) -> None:
        """Recalculates speed, and checks and adjusts if the particle is considered hot or not depending on its speed compared to the threshhold."""
        self._speed = np.linalg.norm(self.velocity)
        self.isHot: bool = True if abs(self._speed) >= SPEED_THRESHHOLD else False

    def _bounce(self, coordinate: int) -> None:
        """Inverts the particles velocity at a cartesian coordinate and recalculates the particle's angle."""
        self.velocity[coordinate] *= -1
        self._speed = np.linalg.norm(self.velocity)
        self._calculateIsHot()
        
    def _isAtMiddleX(self, position: float) -> bool:
        """Determines whether the particle is close to or at the centre of the container based on its position and velocity."""
        middle = CONTAINER_COORDINATES[X] / 2
        if abs(position - middle) <= self.radius:
            return True
        else:
            return False

    def _isAtGateY(self, position: float) -> bool:
        """Determines whether a particle is at the gate where it can cross to the other container."""
        # below gate
        if (
            0.5 * (CONTAINER_COORDINATES[Y] + GATE_SIZE)
            <= position + self.radius
            <= CONTAINER_COORDINATES[Y]
        ):
            return False
        # above gate
        elif (
            0 <= position + self.radius <= 0.5 * (CONTAINER_COORDINATES[Y] - GATE_SIZE)
        ):
            return False
        else:
            return True

    def isOnCorrectSide(self) -> bool:
        """Determines whether a particle is on the correct side of the container based on isHot boolean."""
        # hot should be left
        # cold sohuld be right
        middle = CONTAINER_COORDINATES[X] / 2
        if self.position[X] > middle and self.isHot:
            return False
        elif self.position[X] < middle and self.isHot:
            return True
        elif self.position[X] > middle and not self.isHot:
            return True
        elif self.position[X] < middle and not self.isHot:
            return False
        else:
            raise RuntimeError("Where are we supposed to be?")

    def _hasBreachedBoundary(self, container: int, point: float) -> bool:
        """Determines whether the particle has breached a specific boundary. If it's completely outside, try and push it back in."""
        if point + self.radius >= container:
            # breach on right side
            self._isStuck += 1
            if self._isStuck > 1:
                self.position -= 2*self.radius
            return True
        elif point - self.radius <= 0:
            # breach on left side
            self._isStuck += 1
            if self._isStuck > 1:
                self.position += 2*self.radius
            
            return True
        else:
            # no breach
            self._isStuck = 0
            return False

    def collide(self, other) -> None:
        """Calculates collisions between particles if they are close enough and updates their attributes."""
        distance = np.linalg.norm(self.position - other.position)
        if distance <= self.radius*2.5:
            self.velocity, other.velocity = other.velocity, self.velocity
            self._calculateIsHot()
            other._calculateIsHot()

    def update(self) -> None:
        """Updates the particle's new position based on its previous position and its velocity."""
        self.position += self.velocity

        if self._hasBreachedBoundary(CONTAINER_COORDINATES[X], self.position[X]):
            self._bounce(X)

        if self._hasBreachedBoundary(CONTAINER_COORDINATES[Y], self.position[Y]):
            self._bounce(Y)

        if self._isAtMiddleX(self.position[X]):
            if self._isAtGateY(self.position[Y]):
                # check to see if we should pass through
                if self.isHot and self.velocity[X] > 0:
                    # hot is correct side
                    self._bounce(X)
                elif not self.isHot and self.velocity[X] < 0:
                    # cold is on correct side
                    self._bounce(X)
                else:
                    pass
            else:
                self._bounce(X)

## Generate Molecules

In [2]:
molecules = []
N_PARTICLES = 50
ARBITRARY_MULTIPLIER = 5
for _ in range(N_PARTICLES):
    molecules.append(
        Particle(
            # uniformly distribute particles around the container
            position=np.array([
                np.random.uniform(2*PARTICLE_RADIUS, CONTAINER_COORDINATES[X] - 2*PARTICLE_RADIUS + 1),
                np.random.uniform(2*PARTICLE_RADIUS, CONTAINER_COORDINATES[Y] - 2*PARTICLE_RADIUS + 1),
            ]),
            # multiply a random 2d array and then choose a random direction
            velocity=ARBITRARY_MULTIPLIER * np.random.random(2) * np.random.choice([-1, 1], size=2),
            radius=PARTICLE_RADIUS
        )
    )

## Helper Functions + Numba
Numba is a library that allows us to compile some of the code. The result is that the compile code ends up being run much faster. Since these calculations are being done for every frame when drawing the canvas, it makes sense for us to try and speed things up where possible.

In [3]:
from numba import njit, jit


@jit(nopython=False, forceobj=True)
def get_velocities(molecules: list) -> np.ndarray:
    """Returns the velocities of all particles left of the gate and all particles to the right of the gate."""
    molecule_positions = np.array([m.position for m in molecules])
    molecule_velocities = np.array([m.velocity for m in molecules])
    
    x_positions = molecule_positions[:, 0]
    
    left_velocities = molecule_velocities[x_positions < CONTAINER_COORDINATES[X]/2]
    right_velocities = molecule_velocities[x_positions > CONTAINER_COORDINATES[X]/2]
    
    return np.array(left_velocities), np.array(right_velocities)

@njit
def calculate_kinetic_energy(molecule_velocities: np.ndarray, molecule_masses: float=1.0) -> float:
    KE = 0.0
    for v in molecule_velocities:
        KE += 0.5 * molecule_masses * np.linalg.norm(v)**2
    return KE
        

@njit
def calculate_temperature(kinetic_energy: float) -> float:
    boltzmann = 1.380649e-23
    return (2 / 3) * (kinetic_energy / boltzmann)

## Draw Canvas

In [4]:
from ipycanvas import hold_canvas, Canvas


# NOTE ipycanvas seems to have a bug where numbers must be converted to python's int
# this happens despite the fact that the documentation claims it should be compatible with numpy
cc = Canvas(width=CONTAINER_COORDINATES[X], height=CONTAINER_COORDINATES[Y] + 150)
container_middle = int(CONTAINER_COORDINATES[X]/2), int(CONTAINER_COORDINATES[Y]/2)
STEPS = 2_000
with hold_canvas(cc):
    cc.font = "22px serif"
    left_velocities, right_velocities = None, None  # these have to be initialised for later
    for _ in range(STEPS):
        cc.clear()
        
        # box
        cc.stroke_style = "gray"
        cc.stroke_rect(1, 1, int(CONTAINER_COORDINATES[X]-1), int(CONTAINER_COORDINATES[Y]-1))
        # middle line
        cc.stroke_line(
            container_middle[X], 0, 
            container_middle[X], int(container_middle[Y] - GATE_SIZE/2)
        )
        cc.stroke_line(
            container_middle[X], int(CONTAINER_COORDINATES[Y]),
            container_middle[X], int(container_middle[Y] + GATE_SIZE/2)
        )
        # gate/daemon
        cc.stroke_style = "green"
        cc.stroke_line(
            container_middle[X], int(container_middle[Y] - GATE_SIZE/2),
            container_middle[X], int(container_middle[Y] + GATE_SIZE/2)
        )
        
        for m in molecules:
            m.update()
            for p in molecules:
                m.collide(p)
            if m.isHot:
                cc.fill_style = "red"
            else:
                cc.fill_style = "blue"
            # draw molecule
            cc.fill_circle(int(m.position[X]), int(m.position[Y]), PARTICLE_RADIUS)
        
        left_velocities, right_velocities = get_velocities(molecules)
        
        # hot half
        kinetic_energy_left = calculate_kinetic_energy(left_velocities)
        left_temperature = calculate_temperature(kinetic_energy_left)
        cc.fill_style = "red"
        cc.fill_text(
            f"Energy: {kinetic_energy_left:.3e}J", 
            int(CONTAINER_COORDINATES[X]/12), 
            int(CONTAINER_COORDINATES[Y] + 40)
        )
        cc.fill_text(
            f"\nTemperature: {left_temperature:.3e}k",
            int(CONTAINER_COORDINATES[X]/10 - 60), 
            int(CONTAINER_COORDINATES[Y] + 90)
        )
        cc.fill_text(
            "Hot Side",
            int(CONTAINER_COORDINATES[X]/10 + 50), 
            int(CONTAINER_COORDINATES[Y] + 140)
        )
        
        # cold half
        kinetic_energy_right = calculate_kinetic_energy(right_velocities)
        right_temperature = calculate_temperature(kinetic_energy_right)
        cc.fill_style = "blue"
        cc.fill_text(
            f"Energy: {kinetic_energy_right:.3e}J", 
            int(5*CONTAINER_COORDINATES[X]/8), 
            int(CONTAINER_COORDINATES[Y] + 40)
        )
        cc.fill_text(
            f"\nTemperature: {right_temperature:.3e}k",
            int(7*CONTAINER_COORDINATES[X]/12 - 30), 
            int(CONTAINER_COORDINATES[Y] + 90)
        )
        cc.fill_text(
            "Cold Side",
            int(7*CONTAINER_COORDINATES[X]/12 + 90), 
            int(CONTAINER_COORDINATES[Y] + 140)
        )
        cc.sleep(20)
display(cc)

Canvas(height=750, width=800)

# Findings
As expected, the temperature and the kinetic energy on the left side, the hot side, increase as time goes on while on the right side both decrease. The opening and closing of this gate is not possible without adding extra energy to the system, but in this theoretical example the daemon opens and closes this gate without adding energy to the system and thus we were able to not only order the system, but also decrease entropy. Unfortunately this isn't possible in reality as the opening and closing of the gate requires adding energy to the system.